In [1]:
from tensorflow.examples.tutorials.mnist import mnist, input_data

data_sets = input_data.read_data_sets("/tmp/data/", one_hot=True)

import tensorflow as tf


/home/foriequal0/.local/share/virtualenvs/tf-tutorial-pgL6-UTG/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Extracting /tmp/data/train-images-idx3-ubyte.gz


Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
learning_rate = 0.001
batch_size = 100
display_step = 1
model_path = "/tmp/model.ckpt"

n_hidden_1 = 256
n_hidden_2 = 256
n_input = mnist.IMAGE_PIXELS
n_classes = mnist.NUM_CLASSES

x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])

def multilayer_perceptron(x, weights, biases):
    layer_1 = tf.add(tf.matmul(x, weights[1]), biases[1])
    layer_1 = tf.nn.relu(layer_1)
    
    layer_2 = tf.add(tf.matmul(layer_1, weights[2]), biases[2])
    layer_2 = tf.nn.relu(layer_2)
    
    out_layer = tf.add(tf.matmul(layer_2, weights['out']), biases['out'])
    return out_layer

weights = {
    1: tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    2: tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}

biases = {
    1: tf.Variable(tf.random_normal([n_hidden_1])),
    2: tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

pred = multilayer_perceptron(x, weights, biases)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
training_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

init = tf.global_variables_initializer()

In [3]:
saver = tf.train.Saver()

In [5]:
print("Starting 1st session")
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(3):
        avg_cost = 0.
        total_batch = int(data_sets.train.num_examples / batch_size)
        
        for i in range(total_batch):
            batch_x, batch_y = data_sets.train.next_batch(batch_size)
            _, c = sess.run([training_op, cost],
                            feed_dict={
                                x: batch_x,
                                y: batch_y,
                            })
            
            avg_cost += c / total_batch
        if epoch % display_step == 0:
            print("Epoch: {:04d}, cost={:.9f}".format(epoch+1, avg_cost))
    
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy: ", accuracy.eval({
        x: data_sets.test.images, 
        y: data_sets.test.labels
    }))
    
    save_path = saver.save(sess, model_path)
    print("Model saved in file", save_path)

Starting 1st session


Epoch: 0001, cost=161.572382778


Epoch: 0002, cost=39.883590680


Epoch: 0003, cost=25.089025474
Accuracy:  0.9126


Model saved in file /tmp/model.ckpt


In [7]:
print("Starting 2nd session")
with tf.Session() as sess:
    sess.run(init)
    
    saver.restore(sess, model_path)
    print("Model restored from file: ", save_path)
    
    for epoch in range(4, 10):
        avg_cost = 0.
        total_batch = int(data_sets.train.num_examples / batch_size)
        
        for i in range(total_batch):
            batch_x, batch_y = data_sets.train.next_batch(batch_size)
            _, c = sess.run([training_op, cost],
                            feed_dict={
                                x: batch_x,
                                y: batch_y,
                            })
            
            avg_cost += c / total_batch
        if epoch % display_step == 0:
            print("Epoch: {:04d}, cost={:.9f}".format(epoch+1, avg_cost))
    
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy: ", accuracy.eval({
        x: data_sets.test.images, 
        y: data_sets.test.labels
    }))
    

Starting 2nd session
INFO:tensorflow:Restoring parameters from /tmp/model.ckpt


Model restored from file:  None


Epoch: 0005, cost=17.669635856


Epoch: 0006, cost=12.756952991


Epoch: 0007, cost=9.457247410


Epoch: 0008, cost=6.961526182


Epoch: 0009, cost=5.247319114


Epoch: 0010, cost=3.948492955
Accuracy:  0.9363
